In [ ]:
import logging
import openai
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from telegram import Bot
import random
from spellchecker import SpellChecker

In [ ]:
spell = SpellChecker()

In [ ]:
TOKEN_bot = '6200850293:AAHGb8yb9rXzp3HMlT_NALpqD6oA8'
openai.api_key = 'sk-X9zzdSlfxL8yT3BlbkFJ70pPdp8HNg9iNwIE'

bot = Bot(token=TOKEN)
updater = Updater(bot=bot, use_context=True)

In [ ]:
# Fungsi yang akan dieksekusi saat command /start diterima
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Hello! I am an Indonesian Recipe Chatbot.")

In [ ]:
# Fungsi yang akan dieksekusi saat user_input memiliki penyakit didalam list
def has_medical_condition(user_input):
    # Daftar penyakit yang tidak boleh mengonsumsi makanan tertentu
    forbidden_diseases = {
        "diabetes": ["sugar", "cake", "ice cream"],
        "hypertension": ["salt", "fried food", "processed meat"],
        "celiac disease": ["gluten", "wheat", "barley"],
        "lactose intolerance": ["milk", "cheese", "yogurt"],
        "peanut allergy": ["peanuts", "peanut butter", "peanut oil"],
        "tree nut allergy": ["almonds", "cashews", "walnuts"],
        "shellfish allergy": ["shrimp", "crab", "lobster"],
        "egg allergy": ["eggs", "mayonnaise", "custard"],
        "soy allergy": ["soybeans", "tofu", "soy sauce"],
        "fish allergy": ["salmon", "tuna", "sardines"],
        "wheat allergy": ["bread", "pasta", "cookies"],
        "celiac disease": ["gluten", "wheat", "barley"],
        "heart disease": ["red meat", "butter", "deep-fried food"],
        "kidney disease": ["processed meats", "soda", "high-sodium foods"],
        "liver disease": ["alcohol", "fried food", "high-fat dairy"],
        "asthma": ["dairy products", "eggs", "preservatives"],
        "celiac disease": ["gluten", "wheat", "barley"],
        "gout": ["organ meats", "shellfish", "high-fructose corn syrup"],
    }

    # Mengubah teks masukan menjadi lowercase untuk pengecekan yang tidak case-sensitive
    user_input = user_input.lower()

    # Melakukan pengecekan riwayat penyakit
    for disease, forbidden_foods in forbidden_diseases.items():
        # Jika penyakit ditemukan dalam teks masukan
        if disease in user_input:
            # Mencari makanan terlarang yang terkait dengan penyakit tersebut
            forbidden_food_list = ", ".join(forbidden_foods)
            response = f"I'm sorry, you have a history of {disease}. You are not allowed to consume foods such as {forbidden_food_list}."
            return response

    # Jika tidak ada riwayat penyakit yang ditemukan
    return None

In [ ]:
# Fungsi yang akan dieksekusi saat menerima pesan teks
def reply_message(update, context):
    
    user_input = update.message.text.lower()
    
    # Memeriksa riwayat penyakit
    if has_medical_condition(user_input):
        response = "I'm sorry, you have a history of certain medical conditions that are not allowed to consume this food."
    else:
        # Memperbaiki ejaan dalam teks masukan
        corrected_text = []
        words = user_input.split()
        for word in words:
            corrected_word = spell.correction(word)
            corrected_text.append(corrected_word)

        corrected_input = ' '.join(corrected_text)

    # Logika untuk merespons pertanyaan atau perintah khusus dari pengguna
    if 'hi' in user_input or 'halo' in user_input or 'hello' in user_input:
        response = 'Halo! how can i help you?'
    elif 'thank you' in user_input or 'Thank you' in user_input:
        response = "You're welcome! If you have any questions, feel free to ask me."
    else:
        # Mengirim permintaan ke OpenAI API untuk mendapatkan respons
        response = openai.Completion.create(
            engine='text-davinci-003',
            prompt=corrected_input,
            temperature=0.7,
            max_tokens=1000,
            n=1,
            stop=None,
            frequency_penalty=0,
            presence_penalty=0
        ).choices[0].text.strip()

    # Mengirim respons dari chatbot ke pengguna
    context.bot.send_message(chat_id=update.effective_chat.id, text=response)   

In [ ]:
def main():
    # Inisialisasi updater
    updater = Updater(token=TOKEN, use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # Register handler untuk command /start
    start_handler = CommandHandler('start', start)
    dispatcher.add_handler(start_handler)

    # Register handler untuk pesan teks
    message_handler = MessageHandler(Filters.text & (~Filters.command), reply_message)
    dispatcher.add_handler(message_handler)

    # Jalankan bot
    updater.start_polling()
    updater.idle()

In [ ]:
if __name__ == '__main__':
    logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
    main()